In [ ]:
import json
from pathlib import Path
import pickle

import pandas
from matplotlib import pyplot
from pandas import DataFrame
import yaml


def log_as_dataframe(log):
    columns = ['timestamp', 'level', 'source', 'function', 'data']
    df = pandas.DataFrame(log, columns=columns)
    return df


df = log_as_dataframe(pickle.load(open('../log.pkl', 'rb')))
df = df[df['level'] == 'DATA']
df.head()

In [ ]:
data = []
for row in df.itertuples():
    try:
        value = yaml.load(row.data)
    except yaml.error.YAMLError:
        continue
    if len(value) != 12:
        print(value)
        continue
    data.append([row.timestamp] + value)
df = DataFrame(data)
df.columns = ['timestamp', 'front_left', 'front_right', 'side_left', 'side_right',
              'linear_ideal', 'linear_measured', 'angular_ideal', 'angular_measured',
              'voltage_left', 'voltage_right', 'pwm_left', 'pwm_right']
df['timestamp'] /= 1000
df = df.set_index('timestamp')
df = df.sort_index()
df.head()

## Linear speed feed-forward

In [ ]:
df['time'] = df.index
df['acceleration'] = df['linear_ideal'].diff() / df['time'].diff()
df['force'] = df['acceleration'] / 2. * 0.115

In [ ]:
linear = df[(df['acceleration'] == 0) & (df['angular_ideal'] == 0) & (df['linear_ideal'] != 0)]

In [ ]:
from sklearn import linear_model

def fit_motor_linear_speed(data, y_label):
    X = numpy.expand_dims(data['linear_measured'].values, axis=1)
    y = numpy.expand_dims(data[y_label].values, axis=1)
    regr = linear_model.LinearRegression(fit_intercept=True)
    model = regr.fit(X, y)
    
    line = numpy.array([[data['linear_measured'].min() * 1.2], [data['linear_measured'].max() * 1.2]])
    pyplot.figure()
    pyplot.style.use('seaborn')
    pyplot.xlabel('Speed (m/s)')
    pyplot.ylabel('Voltage (V)')
    pyplot.scatter(x=data['linear_measured'], y=data[y_label], alpha=0.05)
    pyplot.plot(line, model.predict(line), color='red')
    pyplot.show()
    
    return model.intercept_, model.coef_

left_0, left_1 = fit_motor_linear_speed(linear, 'voltage_left')
left_0, left_1

In [ ]:
right_0, right_1 = fit_motor_linear_speed(linear, 'voltage_right')
right_0, right_1

## Force term

In [ ]:
df['voltage_force_left'] = df['voltage_left'] - left_0[0] - left_1[0][0] * df['linear_measured']
df['voltage_force_right'] = df['voltage_right'] - right_0[0] - right_1[0][0] * df['linear_measured']

In [ ]:
force = df[(df['acceleration'] != 0) & (df['angular_ideal'] == 0) & (df['linear_ideal'] != 0)]

In [ ]:
from sklearn.metrics import log_loss

def fit_motor_force_feedforward(data, y_label):
    data = data.dropna()
    X = numpy.expand_dims(data['force'].values, axis=1)
    y = numpy.expand_dims(data[y_label].values, axis=1)
    model = linear_model.LinearRegression(fit_intercept=False).fit(X, y)
    
    line = numpy.array([[data['force'].min() * 1.2], [data['force'].max() * 1.2]])
    pyplot.figure()
    pyplot.style.use('seaborn')
    pyplot.xlabel('Force (N)')
    pyplot.ylabel('Voltage (V)')
    pyplot.scatter(x=data['force'], y=data[y_label], alpha=0.05)
    pyplot.plot(line, model.predict(line), color='red')
    pyplot.show()
    
    return model.coef_[0][0]

fit_motor_force_feedforward(force, 'voltage_force_left')

In [ ]:
fit_motor_force_feedforward(force, 'voltage_force_right')